In [28]:
import wget
import pandas as pd
import sys
import os
import time
import gc
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [3]:
wget.download('https://img-ys01.didistatic.com/static/gaia_datafile/20201012150828attr.txt?expire=1603381254&signiture=UCFOP2nbNnlrnTxv4BWGvbQrS--qbSyW1EqoyivogTI=')
wget.download('https://img-ys01.didistatic.com/static/gaia_datafile/20201012151101topo.txt?expire=1603381254&signiture=pYpxwrnSKTz2bGenqe2GcUmJNwhIMgYJIyY6kNG0uU4=')
wget.download('https://img-ys01.didistatic.com/static/gaia_datafile/20201015181036traffic-fix.tar.gz?expire=1603381254&signiture=ePrSGEPgPxbAFuSG3OsogyUWtihwJ2gYsl3G_jjnBfo=')
wget.download('http://datafountain.int-yt.com/Files/BDCI2020/466DiDi/submit_example.csv')
wget.download('http://datafountain.int-yt.com/Files/BDCI2020/466DiDi/test.zip')

'traffic-fix.tar.gz'

In [4]:
!tar -zxf traffic-fix.tar.gz
!rm traffic-fix.tar.gz
!unzip test.zip
!rm test.zip

In [5]:
attr = pd.read_csv('attr.txt', sep='\t', header=None, 
                   names=['link_id', 'length', 'direction', 'path_class', 
                          'speed_class', 'lane_num', 'speed_limit', 'level', 'width'])
attr

,link_id,length,direction,path_class,speed_class,lane_num,speed_limit,level,width
0,0,19,1,5,7,1,4.166667,5,30
1,1,19,1,5,7,1,4.166667,5,30
2,2,16,1,5,7,1,4.166667,5,30
3,3,16,1,5,7,1,4.166667,5,30
4,4,17,1,5,7,1,4.166667,5,30
...,...,...,...,...,...,...,...,...,...
686100,686100,8,2,4,6,2,13.888889,4,55
686101,686101,37,2,5,7,1,4.166667,5,30
686102,686102,12,2,5,7,1,4.166667,5,30
686103,686103,399,2,3,6,2,13.888889,3,55


In [7]:
attr.to_csv('attr.csv', index=False)

In [8]:
!rm attr.txt

In [7]:
link = pd.read_csv('topo.txt', sep='\t', header=None, 
                   names=['link_id', 'next_link_ids'])
link.sort_values('link_id', ignore_index=True, inplace=True)
link

,link_id,next_link_ids
0,0,1
1,1,"7,2,0"
2,2,3
3,3,"7,2,0"
4,4,"6,5"
...,...,...
683418,686100,649708
683419,686101,686102
683420,686102,685860
683421,686103,686104


In [10]:
link.to_csv('link.csv', index=False)

In [ ]:
with open('edgelist.txt', 'w') as file:
    for x in link.itertuples():
        for nx in x.next_link_ids.split(','):
            file.write('{} {}\n'.format(x.link_id, nx))

In [11]:
!rm topo.txt

In [14]:
date = []

link_id = []
cur_time = []
pred_time = []
label = []

rec_speed_list = []
rec_eta_speed_list = []
rec_car_count_list = []
rec_label_list = []

his_speed_list = []
his_eta_speed_list = []
his_car_count_list = []
his_label_list = []

In [15]:
for path in tqdm(os.listdir('traffic/')):
    if 'txt' not in path:
        continue
    with open('traffic/{}'.format(path), 'r') as file:
        for line in file.readlines():
            date.append(path.rstrip('.txt'))
            contents = line.split(';')
            
            tmp = [int(x) for x in contents[0].split()]
            link_id.append(tmp[0])
            label.append(tmp[1])
            cur_time.append(tmp[2])
            pred_time.append(tmp[3])
            
            tmp = np.array([x.split(':')[1].split(',') for x in contents[1].split()])
            rec_speed_list.append(' '.join(tmp[:,0]))
            rec_eta_speed_list.append(' '.join(tmp[:,1]))
            rec_label_list.append(' '.join(tmp[:, 2]))
            rec_car_count_list.append(' '.join(tmp[:, 3]))
            
            tmp = np.concatenate([np.array([x.split(':')[1].split(',') for x in c.split()]) for c in contents[2:]])
            his_speed_list.append(' '.join(tmp[:,0]))
            his_eta_speed_list.append(' '.join(tmp[:,1]))
            his_label_list.append(' '.join(tmp[:,2]))
            his_car_count_list.append(' '.join(tmp[:,3]))   

100%|██████████| 30/30 [23:15<00:00, 46.50s/it]


In [16]:
traffic_train = pd.DataFrame({
    'date': date,
    'link_id': link_id,
    'cur_time': cur_time,
    'pred_time': pred_time,
    'label': label,
    'rec_speed_list': rec_speed_list,
    'rec_eta_speed_list': rec_eta_speed_list,
    'rec_car_count_list': rec_car_count_list,
    'rec_label_list': rec_label_list,
    'his_speed_list': his_speed_list,
    'his_eta_speed_list': his_eta_speed_list,
    'his_car_count_list': his_car_count_list,
    'his_label_list': his_label_list,
})

In [18]:
traffic_train.sort_values(['date', 'link_id', 'cur_time', 'pred_time'], inplace=True, ignore_index=True)

In [15]:
date = []

link_id = []
cur_time = []
pred_time = []
label = []

rec_speed_list = []
rec_eta_speed_list = []
rec_car_count_list = []
rec_label_list = []

his_speed_list = []
his_eta_speed_list = []
his_car_count_list = []
his_label_list = []

In [16]:
with open('test.txt', 'r') as file:
    for line in file.readlines():
        date.append('20190731')
        contents = line.split(';')

        tmp = [int(x) for x in contents[0].split()]
        link_id.append(tmp[0])
        label.append(tmp[1])
        cur_time.append(tmp[2])
        pred_time.append(tmp[3])
        
        tmp = np.array([x.split(':')[1].split(',') for x in contents[1].split()])
        rec_speed_list.append(' '.join(tmp[:,0]))
        rec_eta_speed_list.append(' '.join(tmp[:,1]))
        rec_label_list.append(' '.join(tmp[:, 2]))
        rec_car_count_list.append(' '.join(tmp[:, 3]))
        
        tmp = np.concatenate([np.array([x.split(':')[1].split(',') for x in c.split()]) for c in contents[2:]])
        his_speed_list.append(' '.join(tmp[:,0]))
        his_eta_speed_list.append(' '.join(tmp[:,1]))
        his_label_list.append(' '.join(tmp[:,2]))
        his_car_count_list.append(' '.join(tmp[:,3]))   

In [17]:
traffic_test = pd.DataFrame({
    'date': date,
    'link_id': link_id,
    'cur_time': cur_time,
    'pred_time': pred_time,
    'label': label,
    'rec_speed_list': rec_speed_list,
    'rec_eta_speed_list': rec_eta_speed_list,
    'rec_car_count_list': rec_car_count_list,
    'rec_label_list': rec_label_list,
    'his_speed_list': his_speed_list,
    'his_eta_speed_list': his_eta_speed_list,
    'his_car_count_list': his_car_count_list,
    'his_label_list': his_label_list,
})

In [18]:
traffic_test.sort_values(['date', 'link_id', 'cur_time', 'pred_time'], inplace=True, ignore_index=True)

In [13]:
traffic_train = pd.read_pickle('traffic_train.pkl')
traffic_test = pd.read_pickle('traffic_test.pkl')

In [14]:
traffic = pd.concat([traffic_train, traffic_test], ignore_index=True)
traffic.head()

,date,link_id,cur_time,pred_time,label,rec_speed_list,rec_eta_speed_list,rec_car_count_list,rec_label_list,his_speed_list,his_eta_speed_list,his_car_count_list,his_label_list
0,20190701,1049,258,288,1,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,43.20 38.20 33.60 27.70 23.50 38.50 38.50 39.4...,23.00 23.00 25.90 22.90 17.60 30.10 30.10 30.1...,1 1 2 3 2 1 1 1 1 1 3 3 3 3 1 1 1 1 2 2,1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 0 0 1 0 0
1,20190701,1049,261,290,1,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,33.60 27.70 23.50 27.80 27.80 39.40 32.60 32.6...,25.90 22.90 17.60 27.70 27.70 30.10 30.10 30.1...,2 3 2 3 3 1 1 1 1 1 3 3 1 1 1 1 2 2 2 2,1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 0 0 3 3
2,20190701,1049,264,288,1,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,43.20 38.20 33.60 27.70 23.50 38.50 38.50 39.4...,23.00 23.00 25.90 22.90 17.60 30.10 30.10 30.1...,1 1 2 3 2 1 1 1 1 1 3 3 3 3 1 1 1 1 2 2,1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 0 0 1 0 0
3,20190701,1049,266,290,1,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,33.60 27.70 23.50 27.80 27.80 39.40 32.60 32.6...,25.90 22.90 17.60 27.70 27.70 30.10 30.10 30.1...,2 3 2 3 3 1 1 1 1 1 3 3 1 1 1 1 2 2 2 2,1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 0 0 3 3
4,20190701,1049,272,290,1,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,0 0 0 0 0,33.60 27.70 23.50 27.80 27.80 39.40 32.60 32.6...,25.90 22.90 17.60 27.70 27.70 30.10 30.10 30.1...,2 3 2 3 3 1 1 1 1 1 3 3 1 1 1 1 2 2 2 2,1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 0 0 3 3


In [15]:
traffic[['rec_speed_{}'.format(i) for i in range(5)]] = traffic['rec_speed_list'].str.split(' ', expand=True).astype('float')
traffic[['rec_eta_speed_{}'.format(i) for i in range(5)]] = traffic['rec_eta_speed_list'].str.split(' ', expand=True).astype('float')
traffic[['rec_car_count_{}'.format(i) for i in range(5)]] = traffic['rec_car_count_list'].str.split(' ', expand=True).astype('int')
traffic[['rec_label_{}'.format(i) for i in range(5)]] = traffic['rec_label_list'].str.split(' ', expand=True).astype('int')

In [18]:
traffic[['his_speed_{}_{}'.format(i, j) for i in range(4) for j in range(5)]] = traffic['his_speed_list'].str.split(' ', expand=True).astype('float')
traffic[['his_eta_speed_{}_{}'.format(i, j) for i in range(4) for j in range(5)]] = traffic['his_eta_speed_list'].str.split(' ', expand=True).astype('float')
traffic[['his_car_count_{}_{}'.format(i, j) for i in range(4) for j in range(5)]] = traffic['his_car_count_list'].str.split(' ', expand=True).astype('int')
traffic[['his_label_{}_{}'.format(i, j) for i in range(4) for j in range(5)]] = traffic['his_label_list'].str.split(' ', expand=True).astype('int')

In [22]:
del traffic['rec_speed_list']
del traffic['rec_eta_speed_list']
del traffic['rec_car_count_list']
del traffic['rec_label_list']
del traffic['his_speed_list']
del traffic['his_eta_speed_list']
del traffic['his_car_count_list']
del traffic['his_label_list']

In [ ]:
label_cols = [x for x in traffic.columns if 'label' in x]
for col in label_cols:
    traffic[col] = traffic[col].replace(4, 3)

In [23]:
traffic.head()

,date,link_id,cur_time,pred_time,label,rec_speed_0,rec_speed_1,rec_speed_2,rec_speed_3,rec_speed_4,rec_eta_speed_0,rec_eta_speed_1,rec_eta_speed_2,rec_eta_speed_3,rec_eta_speed_4,rec_car_count_0,rec_car_count_1,rec_car_count_2,rec_car_count_3,rec_car_count_4,rec_label_0,rec_label_1,rec_label_2,rec_label_3,rec_label_4,his_speed_0_0,his_speed_0_1,his_speed_0_2,his_speed_0_3,his_speed_0_4,his_speed_1_0,his_speed_1_1,his_speed_1_2,his_speed_1_3,his_speed_1_4,his_speed_2_0,his_speed_2_1,his_speed_2_2,his_speed_2_3,his_speed_2_4,his_speed_3_0,his_speed_3_1,his_speed_3_2,his_speed_3_3,his_speed_3_4,his_eta_speed_0_0,his_eta_speed_0_1,his_eta_speed_0_2,his_eta_speed_0_3,his_eta_speed_0_4,his_eta_speed_1_0,his_eta_speed_1_1,his_eta_speed_1_2,his_eta_speed_1_3,his_eta_speed_1_4,his_eta_speed_2_0,his_eta_speed_2_1,his_eta_speed_2_2,his_eta_speed_2_3,his_eta_speed_2_4,his_eta_speed_3_0,his_eta_speed_3_1,his_eta_speed_3_2,his_eta_speed_3_3,his_eta_speed_3_4,his_car_count_0_0,his_car_count_0_1,his_car_count_0_2,his_car_count_0_3,his_car_count_0_4,his_car_count_1_0,his_car_count_1_1,his_car_count_1_2,his_car_count_1_3,his_car_count_1_4,his_car_count_2_0,his_car_count_2_1,his_car_count_2_2,his_car_count_2_3,his_car_count_2_4,his_car_count_3_0,his_car_count_3_1,his_car_count_3_2,his_car_count_3_3,his_car_count_3_4,his_label_0_0,his_label_0_1,his_label_0_2,his_label_0_3,his_label_0_4,his_label_1_0,his_label_1_1,his_label_1_2,his_label_1_3,his_label_1_4,his_label_2_0,his_label_2_1,his_label_2_2,his_label_2_3,his_label_2_4,his_label_3_0,his_label_3_1,his_label_3_2,his_label_3_3,his_label_3_4
0,20190701,1049,258,288,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,43.2,38.2,33.6,27.7,23.5,38.5,38.5,39.4,32.6,32.6,13.4,13.4,13.4,28.9,39.3,30.0,30.0,33.2,30.0,30.0,23.0,23.0,25.9,22.9,17.6,30.1,30.1,30.1,30.1,30.1,11.3,11.3,11.3,11.3,9.6,9.3,9.3,18.9,17.0,17.0,1,1,2,3,2,1,1,1,1,1,3,3,3,3,1,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,2,2,2,1,1,0,0,1,0,0
1,20190701,1049,261,290,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,33.6,27.7,23.5,27.8,27.8,39.4,32.6,32.6,32.6,32.6,13.4,28.9,39.3,39.3,39.3,33.2,30.0,30.0,7.7,7.7,25.9,22.9,17.6,27.7,27.7,30.1,30.1,30.1,30.1,30.1,11.3,11.3,9.6,9.6,9.6,18.9,17.0,17.0,17.0,17.0,2,3,2,3,3,1,1,1,1,1,3,3,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,0,0,3,3
2,20190701,1049,264,288,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,43.2,38.2,33.6,27.7,23.5,38.5,38.5,39.4,32.6,32.6,13.4,13.4,13.4,28.9,39.3,30.0,30.0,33.2,30.0,30.0,23.0,23.0,25.9,22.9,17.6,30.1,30.1,30.1,30.1,30.1,11.3,11.3,11.3,11.3,9.6,9.3,9.3,18.9,17.0,17.0,1,1,2,3,2,1,1,1,1,1,3,3,3,3,1,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,2,2,2,1,1,0,0,1,0,0
3,20190701,1049,266,290,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,33.6,27.7,23.5,27.8,27.8,39.4,32.6,32.6,32.6,32.6,13.4,28.9,39.3,39.3,39.3,33.2,30.0,30.0,7.7,7.7,25.9,22.9,17.6,27.7,27.7,30.1,30.1,30.1,30.1,30.1,11.3,11.3,9.6,9.6,9.6,18.9,17.0,17.0,17.0,17.0,2,3,2,3,3,1,1,1,1,1,3,3,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,0,0,3,3
4,20190701,1049,272,290,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,33.6,27.7,23.5,27.8,27.8,39.4,32.6,32.6,32.6,32.6,13.4,28.9,39.3,39.3,39.3,33.2,30.0,30.0,7.7,7.7,25.9,22.9,17.6,27.7,27.7,30.1,30.1,30.1,30.1,30.1,11.3,11.3,9.6,9.6,9.6,18.9,17.0,17.0,17.0,17.0,2,3,2,3,3,1,1,1,1,1,3,3,1,1,1,1,2,2,2,2,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,0,0,3,3


In [25]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15687465 entries, 0 to 15687464
Columns: 105 entries, date to his_label_3_4
dtypes: float64(50), int64(54), object(1)
memory usage: 12.3+ GB


In [29]:
def reduce_mem(df):
    """降低内存占用"""
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'
          .format(end_mem, 100*(start_mem-end_mem)/start_mem, (time.time()-starttime)/60))
    return df

In [ ]:
from sklearn.preprocessing import LabelEncoder
traffic['date'] = LabelEncoder().fit_transform(traffic.date) + 1

In [30]:
traffic = reduce_mem(traffic)

-- Mem. usage decreased to 2872.46 Mb (77.1% reduction),time spend:4.19 min


In [32]:
traffic.to_pickle('traffic_logs.pkl')

In [ ]:
!rm test.txt
!rm -rf traffic

In [14]:
sub = pd.read_csv('submit_example.csv')
sub.head()

,link,current_slice_id,future_slice_id,label
0,258288,480,500,1
1,70581,378,386,1
2,142513,53,70,3
3,308615,57,59,2
4,72963,380,383,1
